# Running Sum and First Difference Algorithms

### Calculus-like Operations
Convolution can change discrete signals in ways that resemble integration and differentiation. Since the terms "derivative" and "integral" specifically refer to operations on continuous signals, other names are given to their discrete counterparts. The discrete operation that mimics the first derivative is called the first difference. Likewise, the discrete form of the integral is called the running sum. It is also common to hear these operations called the discrete derivative and the discrete integral, although mathematicians frown when they hear these informal terms used.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pickle

In [ ]:
file = {'x':'Signals/InputSignal_f32_1kHz_15kHz.dat'}

x = np.loadtxt(file['x'])
N,M = x.shape
x = x.reshape(N*M, 1)

In [ ]:
plt.plot(x)
plt.title('x[n]')
plt.grid('on')
plt.ylabel('Amplitude')
plt.xlabel('Samples')
plt.show()

## First Difference
This is the discrete version of the first derivative. Each sample in the output signal is equal to the difference between adjacent samples in the input signal. In other words, the output signal is the slope of the input signal.

$$ y[n] = x[n] - x[n-1]$$

In [ ]:
def first_difference(x):
    """ 
        Function that calculates the first difference of an input signal x using the recursive
        equation y[n]=x[n]-x[n-1].
      
        Parameters: 
        x (numpy array): Array of numbers representing the input signal.
      
        Returns: 
        numpy array: Returns first difference of input signal x.
      
        """
    
    N = x.shape[0]
    output = np.zeros(N-1)
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
x_diff = first_difference(x)

assert np.isclose(x_diff, np.diff(x, n=1, axis=0).reshape(-1), atol=0.01).all()

plt.rcParams["figure.figsize"] = (15,5)

plt.subplot(1,2,1)
plt.plot(x)
plt.title('Input Signal')
plt.grid('on')

plt.subplot(1,2,2)
plt.plot(x_diff)
plt.title('First Difference')
plt.grid('on')

## Running Sum
This is the discrete version of the integral. Each sample in the output signal is equal to the sum of all samples in the input signal to the left.

$$ y[n] = x[n] + y[n-1]$$

In [ ]:
def running_sum(x):
    """ 
        Function that calculates the running sum of an input signal x using the recursive
        equation y[n]=x[n]+y[n-1].
      
        Parameters: 
        x (numpy array): Array of numbers representing the input signal.
      
        Returns: 
        numpy array: Returns running sum of input signal x.
      
        """
    N = x.shape[0]
    output = np.zeros(N)
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
x_sum = running_sum(x)

assert np.isclose(x_sum, np.cumsum(x), atol=0.01).all()

plt.rcParams["figure.figsize"] = (15,5)

plt.subplot(1,2,1)
plt.plot(x)
plt.title('Input Signal')
plt.grid('on')

plt.subplot(1,2,2)
plt.plot(x_sum)
plt.title('Running Sum')
plt.grid('on')

## Application: Use your smartphone sensors
In this part we will use the gyroscope and accelerometer of your smartphone to get some data and then we will implement a modified version of the running sum and first difference to these data.

To better understand how data is gathered from these sensors, let's look at the coordinate system of a smarthpone.

<img src="Images/gyroscope.png" alt="smartphone coordinates" width="330" height="400"> 

From the previous image, you can see how standard coordinate system is placed, where as the top of the smartphone points towards the Y coordinate, the X coordinate points to the right,  and the Z coordinate points outside of the screen.

A gyroscope measures the rate of rotation in $rad/s$ around a device's x, y, and z axis. An acceleration sensor measures the acceleration applied to the device, in $m/s^2$, including the force of gravity on each axis. You can read in more detail how these sensors work by reading [here](https://www.maximintegrated.com/en/design/technical-documents/app-notes/5/5830.html)

## Getting the data

1. First you need to download and install [sensorsensei](https://play.google.com/store/apps/details?id=com.sensorsensei) your smartphone, currently only Android devices are supported. 
<br>

2. Then you need to clone the following [repository](https://github.com/priyankark/PhonePi_SampleServer.git) on your computer.
```bash
git clone https://github.com/priyankark/PhonePi_SampleServer.git
```
3. Inside the cloned project go to `Python/Flask` and run `python PhonePi.py`

If you encounter dependencie issues be sure to have both `Flask` and `flask_sockets` installed. You can install them in Anaconda by running:
```bash
 conda install flask==1.1.4
 conda install flask-sockets 
```

When running the `PhonePi.py` script you will create a socket connection between your computer and your smartphone, be sure to connect to the same network. The you can select on your smartphone the type of sensor to use and stream this data to your computer, for our case we will use the accelerometer and gyroscope data, but you can try other sensors as well.

After running the script you will get two files called `accelerometer.txt` and `gyroscope.txt`, these files need to be parsed, therefore a `parse_data` function is provided below. You can just use it to convert the data gathered into a python dictionary.

In [ ]:
def parse_data(file):
    """
    Function that parses a given file with sensor data and returns a dictionary
    of the data inside.
    
    Parameters: 
        file (str): Path of the file to parse.
      
        Returns: 
        dictionary: Returns a dictionary with the `timestamp`, `x`, `y`, and `z` keys
        of the parsed data.
    """
    
    Dictionary = {}
    x = []
    y = []
    z = []
    timestamp = []

    with open(file, "r") as f:
        for line in f:
            for content in line.strip('"{}').split(','):
                key, value = content.split(':')
                Dictionary[key.strip('"')] = value.strip('"')
            x.append(float(Dictionary['x']))
            y.append(float(Dictionary['y']))
            z.append(float(Dictionary['z']))
            timestamp.append(Dictionary['Timestamp'])

    x = np.array(x)
    y = np.array(y)
    z = np.array(z)
    timestamp = np.array(timestamp)
    return {'timestamp':timestamp.astype('int64'), 'x':x.astype('float32'), 
            'y':y.astype('float32'), 'z':z.astype('float32')}

In [ ]:
gyroscope = parse_data('Signals/sensors/gyroscope.txt')

accelerometer = parse_data('Signals/sensors/accelerometer.txt')

## Transform the data
Now that our sensor data is in a suitable format, we need to preprocess it. Remember that the key idea behind these algorithms is that all of them try to run in real time, which means that we prefer to run them as soon as data comes, instead that running them in batches (maybe some exceptions).

The first function that we will modify is the `first_difference` developed earlier. In this case we want to find the total acceleration applied to our smartphone. Data gathered from the accelerometer provides us with both the applied acceleration plus the earth's own acceleration. So the the new function, `process_accelerometer_data` must overcome this limitation. Also, it is important to note that we are interested in finding the **total** acceleration when performing our calculations.

In [ ]:
def process_accelerometer_data(signal, gravity=9.8):
    """ 
        Function that calculates the acceleration observed by an accelerometer inside the
        smartphone. This function must remove the gravity contribution with the high-pass filter 
        exaplained here
        
        https://developer.android.com/guide/topics/sensors/sensors_motion#sensors-motion-accel
      
        Parameters: 
        signal (dictionary): Dictionary with the `timestamp`, `x`, `y`, and `z` keys from
        the gyroscope data.
      
        Returns: 
        numpy array: Returns magnitude of acceleration applied to the system.
      
        """
    
    N = signal['x'].shape[0]
    output = np.zeros(N)
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
t = np.arange(accelerometer['x'].shape[0])/100
accel_process = process_accelerometer_data(accelerometer, gravity=9.8)

with open('accel.pkl', 'rb') as f:
    data_pickle = pickle.load(f)
    
t_pickle = data_pickle[0]
accel_pickle = data_pickle[1]

assert np.isclose(accel_process, accel_pickle, atol=0.01).all()

# Plotting Results
plt.plot(t, accel_process)
plt.grid('on')
plt.xlabel('Time [s]')
plt.ylabel('Acceleration [$m/s^2$]')
plt.show()

In [ ]:
# Now test with your own data
# YOUR CODE HERE
raise NotImplementedError()

Now we will create a function called `process_gyroscope_data` which calculates the rotation of the gyroscope with respect of a given axis. The input signal is the component on the axis of interest. For example, if we want to calculate the rotation on the x axis, we call `process_gyroscope_data(gyroscope['x'])`.

In [ ]:
def process_gyroscope_data(signal_component):
    """ 
        Function that calculates the rotation experienced by a gyroscope inside a smartphone.
        data. Modify the running sum algorithm to include a factor of 180/pi to change rad to 
        degree and the time interval of 100ms when sampling.
        
        You can get some ideas from here
        https://developer.android.com/guide/topics/sensors/sensors_motion#sensors-motion-gyro
        
        Parameters: 
        signal_component (numpy array): Data from the gyroscope from a given axis.
      
        Returns: 
        numpy array: Returns cummulative angular rotation of a given axis.
        """
    
    N = signal_component.shape[0]
    output = np.zeros(N)
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
t = np.arange(gyroscope['x'].shape[0])/100

rot_x = process_gyroscope_data(gyroscope['x'])
rot_y = process_gyroscope_data(gyroscope['y'])
rot_z = process_gyroscope_data(gyroscope['z'])

with open('gyro.pkl', 'rb') as f:
    data_pickle = pickle.load(f)

t_pickle = data_pickle[0]
rot_x_pickle = data_pickle[1][0]
rot_y_pickle = data_pickle[1][1]
rot_z_pickle = data_pickle[1][2]

assert np.isclose(rot_x, rot_x_pickle, atol=0.01).all()
assert np.isclose(rot_y, rot_y_pickle, atol=0.01).all()
assert np.isclose(rot_z, rot_z_pickle, atol=0.01).all()


# Plotting Results
max_val = round(max(rot_x.max(), rot_y.max(), rot_z.max())/360)
min_val = round(min(rot_x.min(), rot_y.min(), rot_z.min())/360)

plt.plot(t, rot_x, label='x-rotation')
plt.plot(t, rot_y, label='y-rotation')
plt.plot(t, rot_z, label='z-rotation')

for i in range(min_val, max_val):
    plt.hlines(360*i, t.min(), t.max(), color='r', alpha=0.6, linestyles='--')

plt.grid('on')
plt.xlabel('Time [s]')
plt.ylabel('Rotation [Degrees]')
plt.legend()
plt.show()

In [ ]:
# Now test with your own data
# YOUR CODE HERE
raise NotImplementedError()

#### References:

* https://www.dspguide.com/ch7/1.htm
* https://www.maximintegrated.com/en/design/technical-documents/app-notes/5/5830.html
* https://developer.android.com/guide/topics/sensors/sensors_motion#sensors-raw-data
* https://github.com/priyankark/PhonePi_SampleServer
* http://www.techbitar.com/sensoduino.html